In [ ]:
!gdown --id 11iR_kDZaJ9IB5-CHVz8DGJ9sLriJWWGEm && unzip -q /content/superai2_pos_tagging.zip && rm /content/superai2_pos_tagging.zip

Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=11iR_kDZaJ9IB5-CHVz8DGJ9sLriJWWGEm 



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q /content/drive/MyDrive/NLP_POS/superai2_pos_tagging.zip

replace __MACOSX/._super-ai-engineer-2021-nlp-pos-tagging? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
path = "/content/super-ai-engineer-2021-nlp-pos-tagging/train/LST20 (Main Resource)/LST20_Corpus/LST20_Corpus/train/"

In [ ]:
import os
import pandas as pd

In [ ]:
dir_list = os.listdir(path)

In [ ]:
dir_list.sort()
dir_list=dir_list[1:]

In [ ]:
word=[]
pos=[]
for file in dir_list:
  file2 = open(path+file, 'r')
  Lines = file2.readlines()
  for line in Lines:
    if line == "\n":
      word.append("_")
      pos.append("PU")
    else:
      word.append(line.split("\t")[0])
      pos.append(line.split("\t")[1])
  file2.close()

In [ ]:
All_data = pd.DataFrame()
All_data["word"]=word
All_data["POS"]=pos

In [ ]:
import gc

In [ ]:
del dir_list,word,pos,Lines
gc.collect()

293

In [ ]:
All_data

,word,POS
0,สุรยุทธ์,NN
1,ยัน,VV
2,ปฏิเสธ,VV
3,ลงนาม,VV
4,_,PU
...,...,...
2778031,ถอน,VV
2778032,รายชื่อ,NN
2778033,ด้วย,PS
2778034,.,PU


In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation

In [ ]:
!pip install -U sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
model_emb = SentenceTransformer('mrp/simcse-model-roberta-base-thai')

In [ ]:
!pip install tqdm

In [ ]:
from tqdm import tqdm

In [ ]:
n=len(All_data)//256

In [ ]:
from numpy import array
from sklearn.decomposition import TruncatedSVD

In [ ]:
emb_list=[]
for i in tqdm(range(n+1)):
  if i==n:
    embeddings = model_emb.encode(list(All_data.iloc[256*i:len(All_data["word"]),0]))
  else:
    embeddings = model_emb.encode(list(All_data.iloc[256*i:(256*(i+1)),0]))
  emb_list=emb_list+list(embeddings)

100%|██████████| 10852/10852 [24:52<00:00,  7.27it/s]


In [ ]:
del embeddings
gc.collect()

306

In [ ]:
len(emb_list[100])

768

In [ ]:
Pos_unique = All_data['POS'].unique()

In [ ]:
Pos_unique=list(Pos_unique)

In [ ]:
Pos_unique

['NN',
 'VV',
 'PU',
 'PS',
 'NG',
 'NU',
 'CL',
 'CC',
 'AX',
 'AV',
 'FX',
 'AJ',
 'PR',
 'PA',
 'IJ',
 'XX']

In [ ]:
def converlabel(Pos_unique,s):
  ind = Pos_unique.index(s)
  return ind

In [ ]:
relabel=[]
for i in All_data['POS']:
  relabel.append(converlabel(Pos_unique,i))

In [ ]:
import random

In [ ]:
new_emb_list = []
for i in tqdm(range(0,len(emb_list),18)):
  if i < len(emb_list)-18:
    temp = emb_list[i:i+18]
  else:
    temp = emb_list[i:len(emb_list)]
    temp0 = [0]*768
    for j in range(18-(len(emb_list)-i)):
      temp.append(temp0)
  new_emb_list.append(temp)

100%|██████████| 154336/154336 [00:00<00:00, 244014.02it/s]


In [ ]:
len(new_emb_list[0])

16

In [ ]:
del emb_list
gc.collect()

88

In [ ]:
del All_data,temp,temp0
gc.collect()

88

In [ ]:
gc.collect()
new_emb_list = np.asarray(new_emb_list,dtype='float32')

In [ ]:
new_emb_list.shape

(173628, 16, 768)

In [ ]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional

In [ ]:
from keras import layers

In [ ]:
!pip install keras-crf

In [ ]:
from keras_crf import CRFModel

In [ ]:
def text_clasifier_lstm(input_shape):


    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    input = Input(shape=input_shape)

    X = Bidirectional(layers.LSTM(256, return_sequences=True,dropout=0.5))(input)
    X = Bidirectional(layers.LSTM(256, activation='relu', return_sequences=True,go_backwards=True,dropout=0.5))(X)
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(X)
    # Add dropout with a probability of 0.1
    X = Dropout(0.3)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128,return_sequences=True,go_backwards=True)(X)
    # Add dropout with a probability of 0.1
    X = Dropout(0.3)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(64)(X)
    # Add a softmax activation
    #X = Activation('softmax')(X)
    #main_lstm = layers.TimeDistributed(Dense(16, activation="relu"))(X)
    base=tf.keras.Model(inputs=input,outputs=X)
    model=CRFModel(base,18)
    # CRF
    #crf = CRF(17)  # CRF layer
    #out = crf(main_lstm)  # output
    # Create Model instance which converts sentence_indices into X.
    #model = Model(input, out)

    return model

In [ ]:
model = text_clasifier_lstm((18,768))

In [ ]:
model.summary()

Model: "crf_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 18, 768)]    0           []                               
                                                                                                  
 bidirectional (Bidirectional)  (None, 18, 512)      2099200     ['input_1[0][0]']                
                                                                                                  
 bidirectional_1 (Bidirectional  (None, 18, 512)     1574912     ['bidirectional[0][0]']          
 )                                                                                                
                                                                                                  
 lstm_2 (LSTM)                  (None, 18, 128)      328192      ['bidirectional_1[0][0]']

In [ ]:
Y_train = []
for i in tqdm(range(0,len(relabel),18)):
  if i < len(relabel)-18:
    temp = relabel[i:i+18]
  else:
    temp = relabel[i:len(relabel)]
    temp0 = 2
    for j in range(18-(len(relabel)-i)):
      temp.append(temp0)
  Y_train.append(temp)

100%|██████████| 154336/154336 [00:00<00:00, 343975.94it/s]


In [ ]:
del temp,temp0,relabel
gc.collect()
Y_train = np.asarray(Y_train,dtype='float32')

In [ ]:
Y_train.shape

(173628, 16)

In [ ]:
path1 = "/content/super-ai-engineer-2021-nlp-pos-tagging/train/LST20 (Main Resource)/LST20_Corpus/LST20_Corpus/eval/"

In [ ]:
dir_list1 = os.listdir(path1)
dir_list1.sort()
dir_list1=dir_list1[1:]

In [ ]:
word1=[]
pos1=[]
for file in dir_list1:
  file3 = open(path1+file, 'r')
  Lines = file3.readlines()
  for line in Lines:
    if line == "\n":
      word1.append("_")
      pos1.append("PU")
    else:
      word1.append(line.split("\t")[0])
      pos1.append(line.split("\t")[1])
  file3.close()

In [ ]:
All_data1 = pd.DataFrame()
All_data1["word"]=word1
All_data1["POS"]=pos1

In [ ]:
All_data1

,word,POS
0,สลด,VV
1,ผัว,NN
2,เมีย,NN
3,เฒ่า,NN
4,ถูก,AX
...,...,...
246475,ขอ,VV
246476,ไว้,AV
246477,ดูแล,VV
246478,เอง,AV


In [ ]:
del dir_list1, word1,pos1
gc.collect()

131

In [ ]:
m=len(All_data1)//256

In [ ]:
emb_list1=[]
for i in tqdm(range(m+1)):
  if i==m:
    embeddings = model_emb.encode(list(All_data1.iloc[256*i:len(All_data1["word"]),0]))
  else:
    embeddings = model_emb.encode(list(All_data1.iloc[256*i:(256*(i+1)),0]))
  emb_list1=emb_list1+list(embeddings)

100%|██████████| 963/963 [01:50<00:00,  8.74it/s]


In [ ]:
len(emb_list1)

246480

In [ ]:
relabel1=[]
for i in All_data1['POS']:
  relabel1.append(converlabel(Pos_unique,i))

In [ ]:
len(relabel1)

246480

In [ ]:
del embeddings
gc.collect()

261

In [ ]:
new_emb_list1 = []
for i in tqdm(range(0,len(emb_list1),18)):
  if len(emb_list1)-i<18:
    temp = emb_list1[i:len(emb_list1)]
    temp0 = [0]*768
    for j in range(18-(len(emb_list1)-i)):
      temp.append(temp0)
    new_emb_list1.append(temp)
  else:
    new_emb_list1.append(emb_list1[i:i+18])

100%|██████████| 13694/13694 [00:00<00:00, 650620.74it/s]


In [ ]:
#del temp,temp0,embeddings,emb_list1
#gc.collect()
new_emb_list1 = np.asarray(new_emb_list1,dtype='float32')

In [ ]:
new_emb_list1.shape

(15405, 16, 768)

In [ ]:
Y_val = []
for i in tqdm(range(0,len(relabel1),18)):
  if len(emb_list1)-i<18:
    temp = relabel1[i:len(relabel1)]
    temp0=0
    for j in range(18-(len(relabel1)-i)):
      temp.append(temp0)
    Y_val.append(temp)
  else:
    Y_val.append(relabel1[i:i+18])

100%|██████████| 13694/13694 [00:00<00:00, 688979.78it/s]


In [ ]:
Y_val=np.asarray(Y_val,dtype='float32')

In [ ]:
Y_val.shape

(15405, 16)

In [ ]:
del temp,temp0
gc.collect()

NameError: ignored

In [ ]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
import keras.backend as K

def f1_metric(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
gc.collect()

1843

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath='/content/drive/MyDrive/NLP_POS/best_model5nnn.h5')
adam = tf.keras.optimizers.Adam(lr=0.0005)
model.compile(optimizer=adam, loss=dice_coef_loss, metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model.fit(new_emb_list, Y_train, epochs = 4, batch_size = 32, shuffle=True,validation_data=(new_emb_list1,Y_val), callbacks=[checkpoint])

Epoch 1/4
4823/4823 [==============================] - 269s 53ms/step - decode_sequence_accuracy: 0.9188 - loss: 4.6010 - val_decode_sequence_accuracy: 0.9554 - val_loss: 3.3356
Epoch 2/4
4823/4823 [==============================] - 253s 52ms/step - decode_sequence_accuracy: 0.9545 - loss: 2.3559 - val_decode_sequence_accuracy: 0.9598 - val_loss: 3.1277
Epoch 3/4
4823/4823 [==============================] - 253s 52ms/step - decode_sequence_accuracy: 0.9578 - loss: 2.8250 - val_decode_sequence_accuracy: 0.9602 - val_loss: 2.7952
Epoch 4/4
4823/4823 [==============================] - 253s 53ms/step - decode_sequence_accuracy: 0.9594 - loss: 3.0120 - val_decode_sequence_accuracy: 0.9613 - val_loss: 4.8338


In [ ]:
!gdown --id 1ZfMnw7YJqLIrbvewDyHnhykfk299fZXW

In [ ]:
model.save('/content/drive/MyDrive/POSTAG/model3.h5')

In [ ]:
path2 = "/content/super-ai-engineer-2021-nlp-pos-tagging/train/LST20 (Main Resource)/LST20_Corpus/LST20_Corpus/test/"

In [ ]:
dir_list2 = os.listdir(path2)
dir_list2.sort()
dir_list2=dir_list2[1:]

In [ ]:
All_data2=pd.DataFrame()
for file in dir_list2:
  data = pd.DataFrame(pd.read_csv(path2+file,sep='\t',header=None,names=['word','POS','NER','boundary'], quotechar=' '))
  All_data2=All_data2.append(data, ignore_index=True)

In [ ]:
All_data2

,word,POS,NER,boundary
0,จีน,NN,B_LOC,B_CLS
1,-,PU,O,I_CLS
2,อินเดีย,NN,B_LOC,I_CLS
3,เสี่ยง,VV,O,I_CLS
4,สูญเสีย,VV,O,I_CLS
...,...,...,...,...
207273,ครหา,VV,O,I_CLS
207274,เกี่ยวกับ,VV,O,I_CLS
207275,ความ,FX,O,I_CLS
207276,ไม่,NG,O,I_CLS


In [ ]:
X_test_indices = sentences_to_indices(All_data2['word'], word2vec, 12)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:32: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [ ]:
relabel2=[]
for i in All_data2['POS']:
  relabel2.append(converlabel(Pos_unique,i))

In [ ]:
Y_test_oh = convert_to_one_hot(relabel2, 16)

In [ ]:
solution = model.predict(X_test_indices)

In [ ]:
def inverselabel(Pos_unique,x):
  return Pos_unique[x]

In [ ]:
result=[]
for i in solution:
  result.append(inverselabel(Pos_unique,np.argmax(i)))

In [ ]:
All_data2["result"]=result

In [ ]:
All_data2

,word,POS,NER,boundary,result
0,จีน,NN,B_LOC,B_CLS,NN
1,-,PU,O,I_CLS,PU
2,อินเดีย,NN,B_LOC,I_CLS,NN
3,เสี่ยง,VV,O,I_CLS,VV
4,สูญเสีย,VV,O,I_CLS,VV
...,...,...,...,...,...
207273,ครหา,VV,O,I_CLS,VV
207274,เกี่ยวกับ,VV,O,I_CLS,VV
207275,ความ,FX,O,I_CLS,FX
207276,ไม่,NG,O,I_CLS,NG


In [ ]:
gc.collect()

2012

In [ ]:
path3 = "/content/drive/MyDrive/NLP_POS/"

In [ ]:
!unzip "/content/drive/MyDrive/NLP_POS/pos_test.txt.zip"

Archive:  /content/drive/MyDrive/NLP_POS/pos_test.txt.zip
replace pos_test.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: pos_test.txt            


In [ ]:
file1 = open('/content/super-ai-engineer-2021-nlp-pos-tagging/pos_test.txt', 'r')
Lines = file1.readlines()

In [ ]:
data_t=[]
for line in Lines:
  data_t.append(line)

In [ ]:
dat=[]
for i in data_t:
  dat.append(i.rstrip("\n"))

In [ ]:
import pandas as pd

In [ ]:
Data3=pd.DataFrame(dat)

In [ ]:
All_data4 = pd.DataFrame(pd.read_csv("/content/super-ai-engineer-2021-nlp-pos-tagging/pos_sample_submission.csv",sep='\t',header=None,names=['word']))

In [ ]:
Data3

,0
0,อย่างไรก็ตาม
1,เครื่อง
2,บิน
3,แอร์บัส
4,
...,...
103909,พร้อม
103910,รับ
103911,การ
103912,โจมตี


In [ ]:
Data3 = Data3.replace('','_')

In [ ]:
Data3

,0
0,อย่างไรก็ตาม
1,เครื่อง
2,บิน
3,แอร์บัส
4,_
...,...
103909,พร้อม
103910,รับ
103911,การ
103912,โจมตี


In [ ]:
m=len(Data3)//256

In [ ]:
m

405

In [ ]:
emb_list3=[]
for i in tqdm(range(m+1)):
  if i==m:
    embeddings = model_emb.encode(list(Data3.iloc[256*i:len(Data3),0]))
  else:
    embeddings = model_emb.encode(list(Data3.iloc[256*i:(256*(i+1)),0]))
  emb_list3=emb_list3+list(embeddings)

100%|██████████| 406/406 [00:42<00:00,  9.50it/s]


In [ ]:
#del Data3
gc.collect()

647

In [ ]:
len(emb_list3)

656

In [ ]:
new_emb_list3 = []
for i in tqdm(range(0,len(emb_list3),18)):
  if i>len(emb_list3)-18:
    temp = emb_list3[i:len(emb_list3)]
    temp0 = [0]*768
    for j in range(18-(len(emb_list3)-i)):
      temp.append(temp0)
    new_emb_list3.append(temp)
  else:
    new_emb_list3.append(emb_list3[i:i+18])

100%|██████████| 5773/5773 [00:00<00:00, 456122.46it/s]


In [ ]:
emb_list3 = np.asarray(new_emb_list3,dtype='float32')

In [ ]:
emb_list3.shape

(656, 16, 768)

In [ ]:
solution_t = model.predict(emb_list3)

In [ ]:
solution_t[0][0]

array([7, 0, 1, 0, 2, 0, 0, 8, 1, 3, 0, 3, 0, 0, 3, 0], dtype=int32)

In [ ]:
solution_arr = [item for sublist in solution_t[0] for item in sublist]

In [ ]:
len(solution_arr)

103920

In [ ]:
result_t=[]
for i in solution_arr:
  result_t.append(inverselabel(Pos_unique,i))

In [ ]:
result_t=result_t[0:103914]

In [ ]:
Data3["result"]=result_t

In [ ]:
Data3

,0,result
0,อย่างไรก็ตาม,CC
1,เครื่อง,NN
2,บิน,VV
3,แอร์บัส,NN
4,_,PU
...,...,...
103909,พร้อม,VV
103910,รับ,VV
103911,การ,FX
103912,โจมตี,VV


In [ ]:
Data3=Data3.drop(103913,axis=0)

In [ ]:
Id = []
for i in range(1,len(Data3)+1):
  Id.append(i)

In [ ]:
len(Id)

103913

In [ ]:
solution1 = dict()
for i in range(len(Id)):
  solution1[Id[i]]=Data3["result"][i]

In [ ]:
solution1

{1: 'CC',
 2: 'NN',
 3: 'VV',
 4: 'NN',
 5: 'PU',
 6: 'NN',
 7: 'NN',
 8: 'AX',
 9: 'VV',
 10: 'PS',
 11: 'NN',
 12: 'PS',
 13: 'NN',
 14: 'NN',
 15: 'PS',
 16: 'NN',
 17: 'PU',
 18: 'NU',
 19: 'PU',
 20: 'CL',
 21: 'CC',
 22: 'VV',
 23: 'NN',
 24: 'VV',
 25: 'AJ',
 26: 'NU',
 27: 'CL',
 28: 'PU',
 29: 'NN',
 30: 'VV',
 31: 'VV',
 32: 'NN',
 33: 'VV',
 34: 'NN',
 35: 'VV',
 36: 'NN',
 37: 'NU',
 38: 'NN',
 39: 'PU',
 40: 'NU',
 41: 'PU',
 42: 'NN',
 43: 'PU',
 44: 'NU',
 45: 'NU',
 46: 'CL',
 47: 'PU',
 48: 'NN',
 49: 'NN',
 50: 'VV',
 51: 'NN',
 52: 'VV',
 53: 'NN',
 54: 'NN',
 55: 'VV',
 56: 'VV',
 57: 'NN',
 58: 'NU',
 59: 'CL',
 60: 'PS',
 61: 'NU',
 62: 'CL',
 63: 'CL',
 64: 'VV',
 65: 'NN',
 66: 'PS',
 67: 'PU',
 68: 'NU',
 69: 'PS',
 70: 'NN',
 71: 'VV',
 72: 'NN',
 73: 'NU',
 74: 'CL',
 75: 'VV',
 76: 'NN',
 77: 'NU',
 78: 'VV',
 79: 'PS',
 80: 'AX',
 81: 'VV',
 82: 'NN',
 83: 'CC',
 84: 'NN',
 85: 'NN',
 86: 'AJ',
 87: 'PU',
 88: 'NN',
 89: 'PU',
 90: 'VV',
 91: 'NN',
 92: 'VV

In [ ]:
#Import CSV
df = pd.read_csv('/content/super-ai-engineer-2021-nlp-pos-tagging/pos_sample_submission.csv')

#Updated
df['Predicted'] = df[["Id","Predicted"]].apply(lambda x : solution1[x["Id"]] if x["Id"] in solution1.keys() else x["Predicted"],axis=1)

#check not null
df["Predicted"].notnull().sum()

103913

In [ ]:
df.to_csv("/content/drive/MyDrive/POSTAG/submission3_final1.csv")

In [ ]:
df

,Id,Predicted
0,1,CC
1,2,NN
2,3,VV
3,4,NN
4,5,PU
...,...,...
103908,103909,VV
103909,103910,VV
103910,103911,VV
103911,103912,FX
